## Link Prediction with GVAE and SEAL for Link prediction

- Variational Graph Auto-Encoders
- Subgraphs, Embeddings, and Attributes

Mateusz Cedro

In [2]:
import torch
!pip install -qU torch-scatter~=2.1.0 torch-sparse~=0.6.16 torch-cluster~=1.6.0 torch-spline-conv~=1.2.1 torch-geometric==2.2.0 -f https://data.pyg.org/whl/torch-{torch.__version__}.html

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 943.4/943.4 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.0/565.0 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
import numpy as np
np.random.seed(0)
import torch
torch.manual_seed(0)
import matplotlib.pyplot as plt
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch import nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
    T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True, split_labels=True, add_negative_train_samples=False),
])

dataset = Planetoid('.', name='Cora', transform=transform)
train_data, val_data, test_data = dataset[0]

Processing...
Done!


## VGAE

In [4]:
from torch_geometric.nn import GCNConv, VGAE

class Encoder(torch.nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()
        self.conv1 = GCNConv(dim_in, 2 * dim_out)
        self.conv_mu = GCNConv(2 * dim_out, dim_out)
        self.conv_logstd = GCNConv(2 * dim_out, dim_out)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

In [5]:
model = VGAE(Encoder(dataset.num_features, 16)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)
    loss = model.recon_loss(z, train_data.pos_edge_label_index) + (1 / train_data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    return float(loss)

@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    return model.test(z, data.pos_edge_label_index, data.neg_edge_label_index)

In [6]:
# Training loop
for epoch in range(301):
    loss = train()
    val_auc, val_ap = test(test_data)
    if epoch % 50 == 0:
        print(f'Epoch {epoch:>3} | Loss: {loss:.4f} | Val AUC: {val_auc:.4f} | Val AP: {val_ap:.4f}')

test_auc, test_ap = test(test_data)
print(f'Test AUC: {test_auc:.4f} | Test AP {test_ap:.4f}')

Epoch  0 | Loss: 3.5139 | Val AUC: 0.6893 | Val AP: 0.7180
Epoch 50 | Loss: 1.3317 | Val AUC: 0.6703 | Val AP: 0.7078
Epoch 100 | Loss: 1.1585 | Val AUC: 0.7163 | Val AP: 0.7355
Epoch 150 | Loss: 1.1033 | Val AUC: 0.7589 | Val AP: 0.7654
Epoch 200 | Loss: 1.0063 | Val AUC: 0.8210 | Val AP: 0.8223
Epoch 250 | Loss: 0.9908 | Val AUC: 0.8229 | Val AP: 0.8253
Epoch 300 | Loss: 0.9916 | Val AUC: 0.8209 | Val AP: 0.8224
Test AUC: 0.8209 | Test AP 0.8224


In [7]:
# Calculate manually the approximated adjacency matrix
z = model.encode(test_data.x, test_data.edge_index)
Ahat = torch.sigmoid(z @ z.T)

In [8]:
Ahat

tensor([[0.6652, 0.6333, 0.7295,  ..., 0.5143, 0.7401, 0.7341],
        [0.6333, 0.7080, 0.7431,  ..., 0.5437, 0.7714, 0.7459],
        [0.7295, 0.7431, 0.8319,  ..., 0.5397, 0.8509, 0.8359],
        ...,
        [0.5143, 0.5437, 0.5397,  ..., 0.5186, 0.5529, 0.5436],
        [0.7401, 0.7714, 0.8509,  ..., 0.5529, 0.8733, 0.8563],
        [0.7341, 0.7459, 0.8359,  ..., 0.5436, 0.8563, 0.8416]],
       grad_fn=<SigmoidBackward0>)

## SEAL
Subgraphs, Embeddings, and Attributes for Link prediction

In [9]:
import numpy as np
from sklearn.metrics import roc_auc_score, average_precision_score
from scipy.sparse.csgraph import shortest_path

import torch.nn.functional as F
from torch.nn import Conv1d, MaxPool1d, Linear, Dropout, BCEWithLogitsLoss

from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, aggr
from torch_geometric.utils import k_hop_subgraph, to_scipy_sparse_matrix

In [10]:
# Load Cora dataset
transform = RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True, split_labels=True)
dataset = Planetoid('.', name='Cora', transform=transform)
train_data, val_data, test_data = dataset[0]
train_data

Data(x=[2708, 1433], edge_index=[2, 8976], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], pos_edge_label=[4488], pos_edge_label_index=[2, 4488], neg_edge_label=[4488], neg_edge_label_index=[2, 4488])

In [11]:
def seal_processing(dataset, edge_label_index, y):
    data_list = []

    for src, dst in edge_label_index.t().tolist():
        sub_nodes, sub_edge_index, mapping, _ = k_hop_subgraph([src, dst], 2, dataset.edge_index, relabel_nodes=True)
        src, dst = mapping.tolist()

        # Remove target link from the subgraph
        mask1 = (sub_edge_index[0] != src) | (sub_edge_index[1] != dst)
        mask2 = (sub_edge_index[0] != dst) | (sub_edge_index[1] != src)
        sub_edge_index = sub_edge_index[:, mask1 & mask2]

        # Double-radius node labeling (DRNL)
        src, dst = (dst, src) if src > dst else (src, dst)
        adj = to_scipy_sparse_matrix(sub_edge_index, num_nodes=sub_nodes.size(0)).tocsr()

        idx = list(range(src)) + list(range(src + 1, adj.shape[0]))
        adj_wo_src = adj[idx, :][:, idx]

        idx = list(range(dst)) + list(range(dst + 1, adj.shape[0]))
        adj_wo_dst = adj[idx, :][:, idx]

        # Calculate the distance between every node and the source target node
        d_src = shortest_path(adj_wo_dst, directed=False, unweighted=True, indices=src)
        d_src = np.insert(d_src, dst, 0, axis=0)
        d_src = torch.from_numpy(d_src)

        # Calculate the distance between every node and the destination target node
        d_dst = shortest_path(adj_wo_src, directed=False, unweighted=True, indices=dst-1)
        d_dst = np.insert(d_dst, src, 0, axis=0)
        d_dst = torch.from_numpy(d_dst)

        # Calculate the label z for each node
        dist = d_src + d_dst
        z = 1 + torch.min(d_src, d_dst) + dist // 2 * (dist // 2 + dist % 2 - 1)
        z[src], z[dst], z[torch.isnan(z)] = 1., 1., 0.
        z = z.to(torch.long)

        # Concatenate node features and one-hot encoded node labels (with a fixed number of classes)
        node_labels = F.one_hot(z, num_classes=200).to(torch.float)
        node_emb = dataset.x[sub_nodes]
        node_x = torch.cat([node_emb, node_labels], dim=1)

        # Create data object
        data = Data(x=node_x, z=z, edge_index=sub_edge_index, y=y)
        data_list.append(data)

    return data_list

In [12]:
# Enclosing subgraphs extraction
train_pos_data_list = seal_processing(train_data, train_data.pos_edge_label_index, 1)
train_neg_data_list = seal_processing(train_data, train_data.neg_edge_label_index, 0)

val_pos_data_list = seal_processing(val_data, val_data.pos_edge_label_index, 1)
val_neg_data_list = seal_processing(val_data, val_data.neg_edge_label_index, 0)

test_pos_data_list = seal_processing(test_data, test_data.pos_edge_label_index, 1)
test_neg_data_list = seal_processing(test_data, test_data.neg_edge_label_index, 0)

In [13]:
# Merge positive and negative data lists to reconstruct the training, validation, and test datasets
train_dataset = train_pos_data_list + train_neg_data_list
val_dataset = val_pos_data_list + val_neg_data_list
test_dataset = test_pos_data_list + test_neg_data_list

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

In [15]:
class DGCNN(torch.nn.Module):
  """Deep-Graph Conv NN
  Args:
    k - number of nodes to hold for each subgraph"""
  def __init__(self, dim_in, k=30):
      super().__init__()

      # GCN layers
      self.gcn1 = GCNConv(dim_in, 32)
      self.gcn2 = GCNConv(32, 32)
      self.gcn3 = GCNConv(32, 32)
      self.gcn4 = GCNConv(32, 1)

      # Global sort pooling
      self.global_pool = aggr.SortAggregation(k=k)

      # Convolutional layers
      self.conv1 = Conv1d(1, 16, 97, 97)
      self.conv2 = Conv1d(16, 32, 5, 1)
      self.maxpool = MaxPool1d(2, 2)

      # Dense layers
      self.linear1 = Linear(352, 128)
      self.dropout = Dropout(0.5)
      self.linear2 = Linear(128, 1)

  def forward(self, x, edge_index, batch):
      # 1. Graph Convolutional Layers
      h1 = self.gcn1(x, edge_index).tanh()
      h2 = self.gcn2(h1, edge_index).tanh()
      h3 = self.gcn3(h2, edge_index).tanh()
      h4 = self.gcn4(h3, edge_index).tanh()
      h = torch.cat([h1, h2, h3, h4], dim=-1)

      # 2. Global sort pooling
      h = self.global_pool(h, batch)

      # 3. Traditional convolutional and dense layers
      h = h.view(h.size(0), 1, h.size(-1))
      h = self.conv1(h).relu()
      h = self.maxpool(h)
      h = self.conv2(h).relu()
      h = h.view(h.size(0), -1)
      h = self.linear1(h).relu()
      h = self.dropout(h)
      h = self.linear2(h).sigmoid()

      return h

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DGCNN(train_dataset[0].num_features).to(device)

optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0001)
loss_fn = BCEWithLogitsLoss()

In [17]:
def train():
    model.train()
    total_loss = 0

    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        loss = loss_fn(out.view(-1), data.y.to(torch.float))
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs

    return total_loss / len(train_dataset)

@torch.no_grad()
def test(loader):
    model.eval()
    y_pred, y_true = [], []

    for data in loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        y_pred.append(out.view(-1).cpu())
        y_true.append(data.y.view(-1).cpu().to(torch.float))

    auc = roc_auc_score(torch.cat(y_true), torch.cat(y_pred))
    ap = average_precision_score(torch.cat(y_true), torch.cat(y_pred))

    return auc, ap

In [18]:
for epoch in range(31):
    loss = train()
    val_auc, val_ap = test(val_loader)
    print(f'Epoch {epoch:>2} | Loss: {loss:.4f} | Val AUC: {val_auc:.4f} | Val AP: {val_ap:.4f}')

test_auc, test_ap = test(test_loader)
print(f'Test AUC: {test_auc:.4f} | Test AP {test_ap:.4f}')

Epoch  0 | Loss: 0.7030 | Val AUC: 0.7088 | Val AP: 0.7261
Epoch  1 | Loss: 0.6401 | Val AUC: 0.7787 | Val AP: 0.8074
Epoch  2 | Loss: 0.6114 | Val AUC: 0.7773 | Val AP: 0.8050
Epoch  3 | Loss: 0.6069 | Val AUC: 0.7790 | Val AP: 0.8094
Epoch  4 | Loss: 0.6051 | Val AUC: 0.7764 | Val AP: 0.8059
Epoch  5 | Loss: 0.6026 | Val AUC: 0.7752 | Val AP: 0.8083
Epoch  6 | Loss: 0.6007 | Val AUC: 0.7722 | Val AP: 0.7996
Epoch  7 | Loss: 0.5994 | Val AUC: 0.7753 | Val AP: 0.8067
Epoch  8 | Loss: 0.5983 | Val AUC: 0.7711 | Val AP: 0.8040
Epoch  9 | Loss: 0.5974 | Val AUC: 0.7731 | Val AP: 0.8075
Epoch 10 | Loss: 0.5965 | Val AUC: 0.7726 | Val AP: 0.7994
Epoch 11 | Loss: 0.5955 | Val AUC: 0.7752 | Val AP: 0.8003
Epoch 12 | Loss: 0.5945 | Val AUC: 0.7739 | Val AP: 0.7967
Epoch 13 | Loss: 0.5929 | Val AUC: 0.7743 | Val AP: 0.7972
Epoch 14 | Loss: 0.5943 | Val AUC: 0.7783 | Val AP: 0.8034
Epoch 15 | Loss: 0.5934 | Val AUC: 0.7833 | Val AP: 0.8134
Epoch 16 | Loss: 0.5926 | Val AUC: 0.7730 | Val AP: 0.79